**Convolutional Neural Network Deep Learning nümunəsi** 
* Hər şeyi bir-bir izah edəcəyəm.
* Uzun və çətin paraqraflar yazmaq əvəzinə, kontenti line by line yazacam.
* Bu dərsliyin sonunda, deep learning-də daha irəli getmək üçün kifayət qədər informasiyaya malik olacaqsınız.
* Bəzi terminlərin öz dilimizdə qarşılığın tapmaqda çətinlik çədiyim üçün ingiliscə yazacağam.
* Kontentə baxaq

<font color='red'>
<br>Məzmun:
* [Giriş](#1)
* [Overview the Data Set](#2)    
* [Build Model](#3)
* [Train the Model](#4)
* [Test model performance](#5)
* [Conclusion](#6)

<a id="1"></a> <br>
# Giriş
* **Deep learning:** Xüsusiyyətləri(features) birbaşa məlumalardan öyrənən Machine Learning texnikalarından biridir.
* **Niyə deep learning?:** Məlumatların çəkisi artdıqca, digər machine learning texnikalari kifayət qədər yaxşı performans göstərə bilmir. Deep Learning bu hallarda daha yaxşı performans və dəqiqlik göstərir.
<a href="http://ibb.co/m2bxcc"><img src="http://preview.ibb.co/d3CEOH/1.png" alt="1" border="0"></a>
* **Deep learning harada istifadə olunur?:** Nitqin tanınması, təsvirlərin(şəkillərin) təsnifatı, Natural Language Processing, Recommendation systems və başqaları.
* **Deep learning və Machine learning arasındakı fərqlər nələrdi?:** 
    * Machine learning deep learning-i əhatə edir. 
    * Machine learning-də xüsusiyytələr(features) manual(əllə) olaraq verilir.
    * Başqa sözlə, Deep learning xüsusiyyətləri(features) birbaşa datadan öyrənir.
<a href="http://ibb.co/f8Epqx"><img src="http://preview.ibb.co/hgpNAx/2.png" alt="2" border="0"></a>

<br>Datamıza baxaq.

In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from tqdm import tqdm, tqdm_notebook
from keras.models import Sequential
import tensorflow.keras.backend as K
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import warnings
# filter warnings
warnings.filterwarnings('ignore')

<a id="2"></a> <br>
# Overview the Data Set
* Biz Aerial photos of Cactus (Neobuxbaumia tetetzo) dataset-indən istifadə edəcəyik (<a href="https://www.kaggle.com/irvingvasquez/cactus-aerial-photos">buradan tanış ola bilərsiniz</a>).
* Burada 17.500 şəkil var.
* Şəkillərdə Kaktus olub olmadığını modelimizə öyrətməliyik.
* Bunun üçün outputda 2 cür cavab olacaq(binary): 0(kaktus yoxdur) və 1(kaktus var).
* Modelimizi öyrətmək üçün datalarımızı hazırlayaq. X şəkil matriksi (cactus və no-cactus) and Y cavablar matriksi (0 and 1).

In [ ]:
train_directory = "../input/train/train/"
train_df = pd.read_csv('../input/train.csv')
train_df.tail()

In [ ]:
fig = plt.figure(figsize=(10, 8))
train_imgs = os.listdir("../input/train/train")
for idx, img in enumerate(np.random.choice(train_imgs, 20)):
    ax = fig.add_subplot(4, 20//4, idx+1, xticks=[], yticks=[])
    im = Image.open("../input/train/train/" + img)
    plt.imshow(im)
    lab = train_df.loc[train_df['id'] == img, 'has_cactus'].values[0]
    ax.set_title(f'Label: {lab}')

In [ ]:
X = []
Y = []
imges = train_df['id'].values
for img_id in tqdm_notebook(imges):
    X.append(cv2.imread(train_directory + img_id))    
    Y.append(train_df[train_df['id'] == img_id]['has_cactus'].values[0])  
X = np.asarray(X)
X = X.astype('float32')
X /= 255  # pixelləri eyni range-ə gətirmək üçün
Y = np.asarray(Y)

<a id="3"></a> <br>
# Build Model
* Artıq X və Y matrislərimiz hazırdır. İndi isə Convolutional Neural Şəbəkəmizi quraq. 
* Bunun üçün Keras kitabxanasından istifadə edəcəyik.

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Dense, BatchNormalization, Activation, GlobalAveragePooling2D, Flatten, Dropout
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.optimizers import Adam

* Convolutional Neural Netvvork haqqında daha dərin öyrənmək üçün <a href="https://www.youtube.com/watch?v=aircAruvnKk">bu linkdən</a> istifadə edə bilərsiniz 

In [ ]:
def simple_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same', use_bias=False, kernel_regularizer=l2(1e-4)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D())

    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=1e-5), 
                  metrics=['accuracy'])
    return model

* CNN-də training zamanı daha dəqiq nəticə almaq üçün Data Augmentationdan istifadə olunur.
* Data Augmentation üçün ImageDataGenerator class-ından istifadə olunur. 
* Data augmentation haqqında ətraflı oxumaq üçün <a href="https://machinelearningmastery.com/image-augmentation-deep-learning-keras">bu linkdən</a> istifadə edin.

In [ ]:
datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset\n",
                             samplewise_center=False,  # set each sample mean to 0\n",
                             featurewise_std_normalization=False,  # divide inputs by dataset std\n",
                             samplewise_std_normalization=False,  # divide each input by its std\n",
                             zca_whitening=False,  # apply ZCA whitening\n",
                             zca_epsilon=1e-06,  # epsilon for ZCA whitening\n",
                             rotation_range=45,  # SET TO 0 IF NEEDED # randomly rotate images in 0 to 180 degrees\n",
                             width_shift_range=0.2,  # randomly shift images horizontally\n",
                             height_shift_range=0.2,  # randomly shift images vertically\n",
                             shear_range=0.2,  # set range for random shear\n",
                             zoom_range=0.2,  # set range for random zoom\n",
                             channel_shift_range=0.,  # set range for random channel shifts\n",
                             fill_mode='nearest',
                             validation_split=0.1,
                             horizontal_flip=True, 
                             vertical_flip=True)  # set mode for filling points outside the input

datagen.fit(X)

<a id="4"></a> <br>
# Train Model
* Modelimizi train etmək üçün  Sequential class-ının <b>fit_generator</b> methodunundan istifadə edirik.

In [ ]:
model = simple_model()
batch_size = 16
epoch = 1000

train = datagen.flow(X, Y, batch_size=batch_size, subset='training')
validate = datagen.flow(X, Y, batch_size=batch_size, subset='validation')

history = model.fit_generator(train, 
                              validation_data=validate,
                              steps_per_epoch=len(X) / batch_size, 
                              epochs=epoch, 
                              shuffle=True, 
                              validation_steps=len(X)/batch_size)

* Training zamani accuracy və loss functionların necə paylanmasına baxaq.

In [ ]:
with open('history.json', 'w') as f:
    json.dump(history.history, f)

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['acc', 'val_acc']].plot()

<a id="5"></a> <br>
# Test model performance
* Test üçün test şəkillərini oxuyub, X_test matris yaradırıq.

In [ ]:
test_directory = "../input/test/test/"
X_test = []
Test_imgs = []
for img_id in tqdm_notebook(os.listdir(test_directory)):
    X_test.append(cv2.imread(test_directory + img_id))     
    Test_imgs.append(img_id)
X_test = np.asarray(X_test)
X_test = X_test.astype('float32')
X_test /= 255

* Train olunmuş modeldən istifadə edərək predict edək.

In [ ]:
# Prediction
test_predictions = model.predict(X_test)

In [ ]:
pred_df = pd.DataFrame(test_predictions, columns=['has_cactus'])
pred_df['id'] = ''
cols = pred_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
pred_df=pred_df[cols]

for i, img in enumerate(Test_imgs):
    pred_df.set_value(i,'id',img)
    
pred_df.to_csv('submission.csv',index=False)    

<a id="6"></a> <br>
# Conclusion
* İlk öncə sona kimi oxuduğunuza görə təşəkkür edirəm.
* Əgər hər hansı səhvlik görsəniz zəhmət olmasa xəbırdar edəsiniz.
* Əgər Machine learning və ya python-dan neyisə başa düşməsəzç aşağıdakı tutoriallardan istifadə edə bilərsiniz.
    * Data Science: https://www.kaggle.com/kanncaa1/data-sciencetutorial-for-beginners
    * Machine learning: https://www.kaggle.com/kanncaa1/machine-learning-tutorial-for-beginners
